Install only required libraries

In [1]:
!pip install bitsandbytes==0.46.1
!pip install triton==3.4.0
!pip install accelerate==0.29.3
!pip install transformers==4.40.0
!pip install peft==0.10.0
!pip install trl==0.8.6

  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached triton-3.5.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
Using cached triton-3.5.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1
  Using cached triton-3.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached triton-3.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (155.6 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling trit

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import torch

In [1]:
#!pip install bitsandbytes==0.43.1
#!pip install transformers==4.40.0
#!pip install peft==0.10.0
#!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Load FLAN-T5-Small in 4-bit

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch

model_name = "google/flan-t5-small"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config, # Temporarily removed due to bitsandbytes installation issue
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Load Dataset and Prepare A Simple Supervised Task

In [8]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts")

def preprocess(batch):
    batch["input_text"] = batch["prompt"]
    batch["target_text"] = ["This is the expected behavior." for _ in batch["prompt"]]
    return batch

dataset = dataset.map(preprocess)

def tokenize_function(examples):
    inputs = [str(ex) for ex in examples["input_text"]]
    targets = [str(ex) for ex in examples["target_text"]]

    # Ensure lists are not empty, as an empty list can sometimes cause TypeErrors in tokenizers backend
    if not inputs or not targets:
        return {"input_ids": [], "attention_mask": [], "labels": []}

    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

tokenized_ds = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_ds["train"]


Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Tokenize for FLAN-T5

In [ ]:
def tokenize_function(examples):
    inputs = [str(ex) for ex in examples["input_text"]]
    targets = [str(ex) for ex in examples["target_text"]]

    # Ensure lists are not empty, as an empty list can sometimes cause TypeErrors in tokenizers backend
    if not inputs or not targets:
        return {"input_ids": [], "attention_mask": [], "labels": []}

    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

tokenized_ds = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_ds["train"]

LoRA Configuration

In [9]:
!pip install --upgrade --no-deps --force-reinstall bitsandbytes==0.43.1
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q", "v"],  # T5 attention
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)


  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1


Training Setup

In [20]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-qlora",
    num_train_epochs=3,
    learning_rate=2e-4,
    auto_find_batch_size=True,
    logging_steps=50,
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # Changed from tokenized_ds to train_dataset
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [15]:
trainer.train()
trainer.model.save_pretrained("./flan-t5-qlora")

Step,Training Loss
50,0.005800


In [24]:
import os

print(os.listdir("./flan-t5-qlora"))


['model.safetensors', 'generation_config.json', 'config.json', 'runs']


In [25]:
trainer.model.save_pretrained("./flan-t5-qlora")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Train & Save


In [26]:
trainer.save_model("./flan-t5-qlora")


In [27]:
trainer.train()
trainer.save_model("./flan-t5-qlora")


Step,Training Loss
50,0.003700


Load Fine-Tuned Model for Inference

In [28]:
from peft import AutoPeftModelForSeq2SeqLM

finetuned_model = AutoPeftModelForSeq2SeqLM.from_pretrained(
    "./flan-t5-qlora",
    device_map="auto",
    quantization_config=bnb_config
)


In [29]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 688,128 || all params: 77,649,280 || trainable%: 0.8862001038515747


Inference Function

In [30]:
def generate(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    output_ids = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generate(finetuned_model, tokenizer, "Explain the value investing and the pros and contras of bitcoin"))


Bitcoin is a cryptocurrency that is a cryptocurrency that is a commodity and is a derivative of the underlying currency.


**COMPARE PRE TRAINED MODEL AND QLORA:**

Load the Pretrained FLAN-T5-Small (baseline)

In [31]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

pretrained_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to("cuda")
pretrained_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


Universal Generate Function

In [32]:
def generate_text(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    output_ids = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


Comparison Function (PRE vs QLoRA)

In [33]:
def compare_models(pretrained, finetuned, tokenizer, prompt):
    print("\n" + "="*120)
    print("📌 PROMPT:")
    print(prompt)
    print("="*120)

    # Pretrained output
    pre_out = generate_text(pretrained, tokenizer, prompt)
    print("\n🤖 PRETRAINED MODEL OUTPUT:\n")
    print(pre_out)

    print("\n" + "-"*120)

    # Fine-tuned output
    ft_out = generate_text(finetuned, tokenizer, prompt)
    print("\n🧠 FINE-TUNED QLoRA MODEL OUTPUT:\n")
    print(ft_out)

    print("\n" + "="*120)

    return pre_out, ft_out


Run the Comparison

In [36]:
prompt = "Explain value investing and the pros and cons of bitcoin."

pre_text, ft_text = compare_models(
    pretrained_model,
    finetuned_model,
    tokenizer,
    prompt
)



📌 PROMPT:
Explain value investing and the pros and cons of bitcoin.

🤖 PRETRAINED MODEL OUTPUT:

Bitcoin is a cryptocurrency that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity that is a commodity

------------------------------------------------------------------------------------------------------------------------

🧠 FINE-TUNED QLoRA MODEL OUTPUT:

Bitcoin is a cryptocurrency that is a commodity and is a currency.

